In [2]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [3]:
#Funcion para arrancar el webdriver
def iniciar_wd(headless=False):
    options=uc.ChromeOptions()
    options.add_argument('--password-store=basic')
    options.add_experimental_option(
        'prefs',
        {'credentials_enable_service':False,
         'profile.password_manager_enabled':False}
    )
    if headless:
        options.add_argument('--headless')

    #Iniciamos el driver
    driver=uc.Chrome(options=options,headless=headless,log_level=3)
    driver.maximize_window()
    
    return driver

In [69]:
def ext_info(driver,url):
    driver.get(url)
    time.sleep(3)
    tit=desc_s=info=evaluaciones=precio=limp=vera=lleg=com=ub=cal=serv=local=''
    
    html=driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    try:
        tit=soup.find('h1').text
    except:
        print('ERROR al obtener el Titulo')
    
    try:
        desc_s=soup.find('h2').text
    except:
        print('Error al obtener la descripcion')
    
    try:
        info=soup.find('ol').text
    except:
        print('Error al obtener la informacion')

    try:
        ev=soup.find_all('span')
        pattern=r'^[0-9]+\s(evaluaciones)$'
        for e in ev:
            if len(re.findall(pattern,e.text))>0:
                evaluaciones=e.text
                break
        
        divs=soup.find_all('div')
        pat_limp=r'^(Limpieza)[0-9],[0-9]$'
        pat_ver=r'^(Veracidad)[0-9],[0-9]$'
        pat_lleg=r'^(Llegada)[0-9],[0-9]$'
        pat_com=r'^(Comunicación)[0-9],[0-9]$'
        pat_ub=r'^(Ubicación)[0-9],[0-9]$'
        pat_cal=r'^(Calidad)[0-9],[0-9]$'

        for d in divs:
            if len(re.findall(pat_limp,d.text))>0:
                limp=d.text
            
            elif len(re.findall(pat_ver,d.text))>0:
                vera=d.text
            elif len(re.findall(pat_lleg,d.text))>0:
                lleg=d.text
            elif len(re.findall(pat_com,d.text))>0:
                com=d.text
            elif len(re.findall(pat_ub,d.text))>0:
                ub=d.text
            elif len(re.findall(pat_cal,d.text))>0:
                cal=d.text
            else:
                pass

    except:
        print('No tiene evaluaciones')
    
    try:
        pattern=r'[0-9]+\s(€)\s$'
        for e in ev:
            if len(re.findall(pattern,e.text))>0:
                precio=e.text
                break
    except:
        print('Error buscando el precio')
    
    sps=soup.find_all('section')

    for s in sps:
        if s.text.find('¿Qué hay en este alojamiento?')!=-1:
            serv=s.text[len('¿Qué hay en este alojamiento?'):]
        
        if s.text.find('¿Dónde me voy a quedar?')!=-1:
            local=s.text[len('¿Dónde me voy a quedar?'):]
        
    return tit,desc_s,info,evaluaciones,precio,limp,vera,lleg,com,ub,cal,serv,local
    

In [70]:
import json

with open('urls.json','r') as jsonfile:
    data=json.load(jsonfile)

driver=iniciar_wd()
ids=[]
titulos=[]
desc_sen=[]
informacion=[]
eval=[]
tipo=[]
precios=[]
enlaces=[]
limpieza=[]
veracidad=[]
llegada=[]
comunicacion=[]
ubicacion=[]
calidad=[]
servicios=[]
localizacion=[]
for k in data.keys():
    for url in data[k]:
        a,b,c,d,f,g,h,i,j,l,m,n,o=ext_info(driver,url)
        
        ids.append(hash(url))
        titulos.append(a)
        desc_sen.append(b)
        informacion.append(c)
        eval.append(d)
        tipo.append(k)
        precios.append(f)
        enlaces.append(url)
        limpieza.append(g)
        veracidad.append(h)
        llegada.append(i)
        comunicacion.append(j)
        ubicacion.append(l)
        calidad.append(m)
        servicios.append(n)
        localizacion.append(o)

driver.close()

In [72]:
result=pd.DataFrame(zip(ids,titulos,desc_sen,informacion,eval,tipo,precios,enlaces,limpieza,veracidad,llegada,comunicacion,ubicacion,calidad,servicios,localizacion),columns=['ID','Titulo','Descripcion Simple','Informacion','Evaluaciones','Tipo','Precio','URL','Limpieza','Veracidad','Llegada','Comunicacion','Ubicacion','Calidad','Servicios','Localizacion'])

,ID,Titulo,Descripcion Simple,Informacion,Evaluaciones,Tipo,Precio,URL,Limpieza,Veracidad,Llegada,Comunicacion,Ubicacion,Calidad,Servicios,Localizacion
0,1200688382914319681,Magníficas vistas sobre la arena de la playa Wifi,Alojamiento entero: piso en Guardamar de la Sa...,6 viajeros · · 3 dormitorios · · 5 camas · ...,59 evaluaciones,A pie de playa,78 €,https://www.airbnb.es/rooms/31709557?adults=1&...,"Limpieza4,9","Veracidad4,9","Llegada5,0","Comunicación4,8","Ubicación4,9","Calidad4,8","Vistas al marVistas al marAcceso a la playa, p...","Guardamar de la Safor, Comunidad Valenciana, E..."
1,5375220911806434689,Melior-1E,"Alojamiento entero: piso en Calp, España",6 viajeros · · 2 dormitorios · · 3 camas · ...,8 evaluaciones,A pie de playa,83 €,https://www.airbnb.es/rooms/37584336?adults=1&...,"Limpieza4,9","Veracidad4,5","Llegada4,5","Comunicación4,8","Ubicación4,8","Calidad4,8",Vistas a la montañaWifiTVAscensorLavadoraAire ...,"Calp, España"
2,556517738340669403,VERALUX,"Alojamiento entero: piso en Benidorm, España",4 viajeros · · 2 dormitorios · · 3 camas · ...,20 evaluaciones,A pie de playa,70 €,https://www.airbnb.es/rooms/46527977?adults=1&...,"Limpieza3,5","Veracidad4,6","Llegada4,7","Comunicación4,8","Ubicación5,0","Calidad4,3","Acceso a la playa, primera líneaCocinaWifiZona...","Benidorm, Comunidad Valenciana, España1ª línea..."
3,8800182335332003755,Isea Calpe,"Alojamiento entero: piso en Calp, España",4 viajeros · · 2 dormitorios · · 3 camas · ...,557 evaluaciones,A pie de playa,128 €,https://www.airbnb.es/rooms/721277134871448378...,,,,,,,CocinaWifiPiscinaTVAscensorLavadoraAire acondi...,"Calp, Comunidad Valenciana, España"
4,-5036294419472545696,"Apartamento luminoso, internet, Netflix",Alojamiento entero: apto. residencial en Grau ...,6 viajeros · · 2 dormitorios · · 6 camas · ...,64 evaluaciones,A pie de playa,85 €,https://www.airbnb.es/rooms/50155790?adults=1&...,"Limpieza4,9","Veracidad4,9","Llegada4,9","Comunicación4,9","Ubicación4,7","Calidad4,7",Vistas al marVistas al jardínAcceso a la playa...,"Grau i Platja, Comunidad Valenciana, España"


In [73]:
result.to_csv('DatasetAirbnb.csv',index=False)